# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config_maps import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../Downloads/Cities.csv")
cities_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Set Humidity as Weight
humidity_df = cities_df["Humidity"].astype(float)


#Set Lat and Lng as Locations
locations_df = cities_df[["Lat", "Lng"]]

In [4]:
fig = gmaps.figure()

heat_df = gmaps.heatmap_layer(locations_df, weights=humidity_df)
fig.add_layer(heat_df)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
#Narrow Cities to Reflect Conditions
narrowed_df = cities_df.loc[(cities_df["Max Temp"] > 80) & (cities_df["Max Temp"] < 70) & (cities_df["Cloudiness"] == 0) & (cities_df["Wind Speed"] < 10),:]

#Drop Null Values
narrowed_df = narrowed_df.dropna(how='any')

#Show DataFrame
narrowed_df.head()



,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed


In [23]:
print ("No cities appear to be in these specific weather conditions")

No cities appear to be in these specific weather conditions


In [27]:
narrowed_df = cities_df.loc[(cities_df["Max Temp"] > 50) & (cities_df["Max Temp"] < 70) & (cities_df["Cloudiness"] == 0) & (cities_df["Wind Speed"] < 10),:]

#Drop Null Values
narrowed_df = narrowed_df.dropna(how='any')

#Show DataFrame
narrowed_df.head(20)

#I change the conditions so i will get some results. 

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
31,31,qingdao,0,CN,1558378760,54,36.09,120.35,59.00,2.24
37,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05
84,84,emerald,0,AU,1558378770,82,-23.53,148.16,60.80,6.93
112,112,bredasdorp,0,ZA,1558378775,62,-34.53,20.04,55.00,8.05
141,141,sanchazi,0,CN,1558378781,82,30.43,103.62,64.40,2.24
168,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17
192,192,port hedland,0,AU,1558378791,51,-20.31,118.58,62.60,6.93
214,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.00,6.93
252,252,kajaani,0,FI,1558378827,62,64.22,27.73,59.00,6.93
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
hotel_df = cities_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,ostrovnoy,RU,68.05,39.51,
1,mahebourg,MU,-20.41,57.70,
2,qaanaaq,GL,77.48,-69.36,
3,zhuhai,CN,40.71,112.04,
4,cape town,ZA,-33.93,18.42,


In [10]:
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index , row in hotel_df.iterrows(): 
    
     lat = cities_df.loc[i]['Lat']
    
     lng = cities_df.loc[i]['Lng']

    params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : gkey}

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get (url, params=params)
    results_df = requested.json()
    
    try: 
        hotel_df.append(results_df["results"][0]['name'])
    except: 
         hotel_df.append("")
cities_df["Hotel Name"] = hotel_df
cities_df = cities_df.dropna(how='any')
cities_df.head()
    
    #I try using iterrows for this part but appears to have some mistake of identation. 

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 11)

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)



In [ ]:
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure